In [7]:
import sys
import pathlib
src_path = pathlib.Path().absolute().parent / "src"
sys.path.append(str(src_path))
import data_functions
import custom_transformers as ct
import pandas as pd
import numpy as np

from sklearn.compose import  make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [8]:
x_train, x_test, y_train = data_functions.get_dataframes()

In [25]:
x_train['construction_year'].value_counts()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [3]:
x_train['region_code'].value_counts()

11    5300
17    5011
12    4639
3     4379
5     4040
18    3324
19    3047
2     3024
16    2816
10    2640
4     2513
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64

In [59]:
#USE QUALITY GROUP INSTEAD OF WATER QUALITY
y_train[x_train['quality_group'] == 'salty']['status_group'].value_counts()

non functional             2504
functional                 2394
functional needs repair     297
Name: status_group, dtype: int64

In [57]:
y_train['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [40]:
x_train['quality_group'].value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

In [14]:
x_train['source_class'].value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

In [61]:
x_train['quantity'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [60]:
x_train['quantity_group'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

In [84]:
other_vals = x_train['installer'].value_counts().index[x_train['installer'].value_counts() < 10]
other_vals

Index(['Kalumbwa', 'Ministry of water engineer', 'ACTIVE TANK CO', 'Redep',
       'Kilwa company', 'European Union', 'Roman Catholic Rulenge Diocese',
       'No', 'CCT', 'REGIONAL WATER ENGINEER ARUSHA',
       ...
       'Mamvua Kakungu', 'ELCA', 'SADIKI KANGELO', 'GERMAN', 'CGI',
       'Village water attendant', 'A.D.B', 'TWIG', 'TCRS/ TWESA',
       'Mgaya Mwita'],
      dtype='object', length=1722)

In [81]:
sum(x_train['installer'].value_counts()[x_train['installer'].value_counts() < 10])

3533

In [74]:
x_train[xtrain['installer'] == x]

1722

In [6]:
sdata = ct.ChooseStrictFeatures()
ldata = ct.ChooseLooseFeatures()
strict_x_train = sdata.transform(x_train)
loose_x_train = ldata.transform(x_train)

In [7]:
bi = ct.BinInstaller()

In [8]:
bined_strict_x_tr = bi.transform(strict_x_train)
bined_loose_x_tr = bi.transform(loose_x_train)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [11]:
bined_loose_x_tr['construction_year'].isna().sum()

0

In [9]:
tcy = ct.TransformConstructionYear(2020)

In [10]:
x_tr_strict_final = tcy.transform(bined_strict_x_tr)
x_tr_loose_final = tcy.transform(bined_loose_x_tr)

In [11]:
x_tr_strict_final.isna().sum()

amount_tsh                   0
gps_height                   0
installer                 3655
basin                        0
region                       0
lga                          0
population                   0
construction_year        20709
extraction_type_group        0
payment_type                 0
quality_group                0
quantity                     0
source_type                  0
waterpoint_type              0
dtype: int64

In [19]:
xtr_testing = x_train.copy()
si_test = SimpleImputer()
transformed_test = si_test.fit_transform(np.array(xtr_testing['construction_year']).reshape(-1,1))

In [22]:
xtr_testing = pd.DataFrame(transformed_test)
xtr_testing.columns = ['construction_year']

In [23]:
test_tcy = ct.TransformConstructionYear(2020)
test_tcy.transform(xtr_testing)

<class 'pandas.core.frame.DataFrame'>


,construction_year
0,21.0
1,10.0
2,11.0
3,34.0
4,NaN
...,...
59395,21.0
59396,24.0
59397,NaN
59398,NaN


In [24]:
xtr_testing.loc[4]

construction_year    0.0
Name: 4, dtype: float64

In [9]:
two = make_column_transformer((SimpleImputer(), ['construction_year']))
three = make_column_transformer((SimpleImputer(missing_values = 0.0), ['construction_year']))
four = make_column_transformer((ct.TransformConstructionYear(2020), ['construction_year']))


In [10]:
alpha = two.fit_transform(x_train)
alpha = pd.DataFrame(alpha, columns = ['construction_year'])
alpha['construction_year'].value_counts()

0.0       20709
2010.0     2645
2008.0     2613
2009.0     2533
2000.0     2091
2007.0     1587
2006.0     1471
2003.0     1286
2011.0     1256
2004.0     1123
2012.0     1084
2002.0     1075
1978.0     1037
1995.0     1014
2005.0     1011
1999.0      979
1998.0      966
1990.0      954
1985.0      945
1980.0      811
1996.0      811
1984.0      779
1982.0      744
1994.0      738
1972.0      708
1974.0      676
1997.0      644
1992.0      640
1993.0      608
2001.0      540
1988.0      521
1983.0      488
1975.0      437
1986.0      434
1976.0      414
1970.0      411
1991.0      324
1989.0      316
1987.0      302
1981.0      238
1977.0      202
1979.0      192
1973.0      184
2013.0      176
1971.0      145
1960.0      102
1967.0       88
1963.0       85
1968.0       77
1969.0       59
1964.0       40
1962.0       30
1961.0       21
1965.0       19
1966.0       17
Name: construction_year, dtype: int64

In [11]:
beta = pd.DataFrame(three.fit_transform(alpha), columns = ['construction_year'])
beta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 1 columns):
construction_year    59400 non-null float64
dtypes: float64(1)
memory usage: 464.2 KB


In [12]:
charlie = four.fit_transform(beta)

<class 'pandas.core.frame.DataFrame'>


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [7]:
column_transform_var.fit_transform(x_train)

<class 'pandas.core.frame.DataFrame'>


TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [3]:
strict_preprocessing = make_pipeline(ct.ChooseStrictFeatures(),
                                    ct.BinInstaller(),
                                    column_transform_var)

#loose_preprocessing = make_pipeline(ct.ChooseLooseFeatures(),
 #                                   ct.BinInstaller(),
  #                                  make_column_transformer((SimpleImputer(), data_functions.get_numeric_features(data_functions.get_loose_features())),
   #                                                           (StandardScaler(), data_functions.get_numeric_features(data_functions.get_loose_features())),
    #                                                          (OneHotEncoder(sparse = False), data_functions.get_categorical_features(data_functions.get_loose_features())),
     #                                                         remainder = 'passthrough'),
      #                              ct.TransformConstructionYear(2020))

In [4]:
strict_model = make_pipeline(strict_preprocessing, DecisionTreeClassifier())

In [5]:
cross_val_score(strict_model, x_train, y_train)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'